# CAR DEKHO

In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\ADMIN\Documents\cardekho.csv")
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0


In [2]:
df.isnull().sum()

name                    0
year                    0
selling_price           0
km_driven               0
fuel                    0
seller_type             0
transmission            0
owner                   0
mileage(km/ltr/kg)    221
engine                221
max_power             215
seats                 221
dtype: int64

In [3]:
df["mileage(km/ltr/kg)"]=df["mileage(km/ltr/kg)"].fillna(df["mileage(km/ltr/kg)"].mean())

In [4]:
df["engine"]=df["engine"].fillna(df["engine"].mean())

In [5]:
df['max_power'] = pd.to_numeric(df['max_power'], errors='coerce')

df["max_power"]=df["max_power"].fillna(df["max_power"].mean())

In [6]:
df.fillna({"seats": df["seats"].mode()[0]}, inplace=True)


In [7]:
df.drop(columns = ['name','seller_type','transmission','seats'],inplace=True)

In [8]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [9]:
encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
col=["fuel","owner"]
df_arr=encoder.fit_transform(df[col])
df_encoder_col=encoder.get_feature_names_out(col)
df_encoded=pd.DataFrame(df_arr,columns=df_encoder_col)
df_new=pd.concat([df.drop(columns=col),df_encoded],axis=1)
df_new.head()

,year,selling_price,km_driven,mileage(km/ltr/kg),engine,max_power,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,23.40,1248.0,74.00,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2014,370000,120000,21.14,1498.0,103.52,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2006,158000,140000,17.70,1497.0,78.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2010,225000,127000,23.00,1396.0,90.00,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2007,130000,120000,16.10,1298.0,88.20,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x=df_new.drop(columns=["selling_price"],axis=1)
y=df_new["selling_price"]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_trf = scaler.fit_transform(x_train)
x_test_trf = scaler.transform(x_test)


In [14]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
x.columns

Index(['year', 'km_driven', 'mileage(km/ltr/kg)', 'engine', 'max_power',
       'fuel_CNG', 'fuel_Diesel', 'fuel_LPG', 'fuel_Petrol',
       'owner_First Owner', 'owner_Fourth & Above Owner', 'owner_Second Owner',
       'owner_Test Drive Car', 'owner_Third Owner'],
      dtype='object')

In [16]:
model = Sequential()

model.add(Dense(64,activation='relu',input_dim=14))
model.add(Dense(32,activation='relu'))
Dropout(0.2)
model.add(Dense(32,activation='relu'))
Dropout(0.2)
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='linear'))

C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,185 (20.25 KB)

 Trainable params: 5,185 (20.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='Adam',loss='mse',metrics=['mae'])

In [19]:
# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the best epoch
)

In [20]:
history = model.fit(x_train_trf,y_train,batch_size=20,epochs=50,verbose=1,validation_split=0.2)#callbacks=[early_stopping])

Epoch 1/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1072884416512.0000 - mae: 639559.3750 - val_loss: 854643638272.0000 - val_mae: 569343.1875
Epoch 2/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 544493993984.0000 - mae: 392998.0625 - val_loss: 235261722624.0000 - val_mae: 283211.0625
Epoch 3/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 265312059392.0000 - mae: 283323.4062 - val_loss: 189808607232.0000 - val_mae: 248402.5781
Epoch 4/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 224305102848.0000 - mae: 256494.5781 - val_loss: 163678306304.0000 - val_mae: 226274.9062
Epoch 5/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 199246528512.0000 - mae: 245422.5312 - val_loss: 147534430208.0000 - val_mae: 212510.0312
Epoch 6/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 182673965056.0000 - mae: 229275.1719 - val_loss: 136463802368.0000 - val_mae: 201473.7656
Epoch 7/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 169405267968.0000 - mae: 221421.8594 - 

In [21]:
from sklearn.metrics import r2_score

In [22]:
y_pred=model.predict(x_test_trf)
r2=r2_score(y_test,y_pred)
r2

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


0.9005215764045715

In [26]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(7,)))  # 👈 change input_shape to 7
# Add more layers as needed
model.add(Dense(units=1))  # Output layer

In [27]:
print(x_train_trf.shape)  # Should be (num_samples, 10)

(6502, 14)


In [29]:
# ✅ Define the model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(14,)))  # match your input shape
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # sigmoid for binary classification

# ✅ Compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ Add early stopping if not defined
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# ✅ Train the model
history = model.fit(x_train_trf, y_train, batch_size=20, epochs=50, verbose=1,
                    validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0000e+00 - loss: -89810256.0000 - val_accuracy: 0.0000e+00 - val_loss: -328947136.0000
Epoch 2/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0000e+00 - loss: -1368648576.0000 - val_accuracy: 0.0000e+00 - val_loss: -2764296960.0000
Epoch 3/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: -6320903168.0000 - val_accuracy: 0.0000e+00 - val_loss: -9775352832.0000
Epoch 4/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: -17174078464.0000 - val_accuracy: 0.0000e+00 - val_loss: -22825740288.0000
Epoch 5/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0000e+00 - loss: -35737432064.0000 - val_accuracy: 0.0000e+00 - val_loss: -43655139328.0000
Epoch 6/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: -63030255616.0000 - val_accuracy: 0.0000e+00 - val_loss: -72658305024.0000
Epoch 7/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accurac